In [1]:
import os
import pandas as pd
import torch

from transformers import AutoTokenizer, T5ForConditionalGeneration, MBartTokenizer, MBartForConditionalGeneration, BertForTokenClassification, AutoModelForSequenceClassification
from nltk import sent_tokenize

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
bloggers = os.listdir('../data/conclusions/')

In [4]:
bloggers

['4pda', 'ixbt', 'mobile-review', 'rozetked', 'wylsacom']

In [5]:
fin_lst = []
df = pd.DataFrame()

for blogger in bloggers:
    file_names = os.listdir(f'../data/conclusions/{blogger}')
    phone_names = [phone_name.split('_')[0] for phone_name in file_names]
    blogger_name_concat = [blogger for _ in range(len(file_names))]
    cur_df = pd.DataFrame([file_names, phone_names, blogger_name_concat],).T
    df = pd.concat([df, cur_df])
    # fin_lst.append([file_names, phone_names, [blogger for _ in range(len(file_names))]])

In [6]:
df.rename(columns={0: 'file_name', 1: 'phone_name', 2: 'blogger'}, inplace=True)

In [7]:
input_file = open('../data/conclusions/4pda/asus zenfone 10_text_20230906.txt', encoding="utf-8")


In [8]:
conclusion_lst = []
full_text_lst = []

for i in range(len(df)):

    
    conclusion_path = f"../data/conclusions/{df.iloc[i, 2]}/{df.iloc[i, 0]}"
    full_text_path = f"../data/reviews/{df.iloc[i, 2]}/{df.iloc[i, 0]}"
    
    conclusion_file = " ".join(open(conclusion_path, encoding="utf-8").readlines())
    full_text_file = " ".join(open(full_text_path, encoding="utf-8").readlines())
    
    conclusion_text = conclusion_file.replace('\n', '').replace('\r', '')
    full_text = full_text_file.replace('\n', '').replace('\r', '')
    
    conclusion_lst.append(conclusion_text)
    full_text_lst.append(full_text)
    
df['conclusion'] = conclusion_lst
df['full_text'] = full_text_lst

In [9]:
df

,file_name,phone_name,blogger,conclusion,full_text
0,asus zenfone 10_text_20230906.txt,asus zenfone 10,4pda,"Как и предшественник, новый ZenFone 10 получи...",Обзор ASUS ZenFone 10: мощный малый глазами за...
1,honor 70_text_20230906.txt,honor 70,4pda,Итоги Honor 70 — интересный смартфон среднего...,Обзор Honor 70: все тренды учтены01.11.2243В р...
2,huawei mate 50 pro_text_20230906.txt,huawei mate 50 pro,4pda,Итоги HUAWEI Mate 50 Pro — достойный представ...,Обзор HUAWEI Mate 50 Pro: в погоне за фотоапп...
3,huawei mate 50_text_20230906.txt,huawei mate 50,4pda,Итоги HUAWEI Mate 50 — неоднозначный аппарат....,Обзор HUAWEI Mate 50: всё на максималках13.12...
4,huawei nova 10 pro_text_20230906.txt,huawei nova 10 pro,4pda,"Смартфон радует хорошей производительностью, ...","Обзор HUAWEI nova 10 Pro: снимай красиво, зар..."
...,...,...,...,...,...
9,nothing phone 2_text_20230906.txt,nothing phone 2,wylsacom,"Несмотря на все перечисленные недостатки, Not...",Так как обзор выходит в момент снятия эм...
10,oneplus 10 pro_text_20230906.txt,oneplus 10 pro,wylsacom,Так стоит ли мечтать о OnePlus 10 Pro или при...,Появление нового флагмана — всегда событие....
11,redmi note 11_text_20230906.txt,redmi note 11,wylsacom,"Пожурить телефон можно за камеру: возможно, к...","Базовый дизайн Мой телефон чёрного цвета,..."
12,tecno pova 5_text_20230906.txt,tecno pova 5,wylsacom,"Когда покупаешь дорогой смартфон, то хочешь э...",Популярная серия смартфонов TECNO POVA пол...


In [10]:
class_tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
class_model = AutoModelForSequenceClassification.from_pretrained(
    "../models/tiny_rubert_fine_tune/tiny_rubert_finetuned_v1", num_labels=6
).to(device)

ext_sum_tokenizer = AutoTokenizer.from_pretrained("IlyaGusev/rubert_ext_sum_gazeta")
sep_token = ext_sum_tokenizer.sep_token
sep_token_id = ext_sum_tokenizer.sep_token_id
ext_sum_model = BertForTokenClassification.from_pretrained("IlyaGusev/rubert_ext_sum_gazeta").to(device)

model_name = "IlyaGusev/mbart_ru_sum_gazeta"
mbart_tokenizer = MBartTokenizer.from_pretrained(model_name)
mbart_model = MBartForConditionalGeneration.from_pretrained(model_name).to(device)

In [11]:
def get_sentances(text):
    paragraphs = []
    for paragraph in text.replace("\r\n", "\n").split("\n\n"):
        if len(paragraph.strip()) > 0:
            paragraphs.append(sent_tokenize(paragraph.strip()))
    
    return [sentance for paragraph in paragraphs for sentance in paragraph]

def get_cat(sentances, model=class_model, tokenizer=class_tokenizer, cats=6):
    weighted_sentances = [{} for i in range(cats)]
    final_sentances = []
    
    for sent in sentances:
        
        inputs = tokenizer(
        sent, padding=True, truncation=True, return_tensors="pt"
        ).to(device)
        logits = model(**inputs).logits
        
        if logits.max().item() < 5:
            continue
        
        pred_cat = logits.argmax().item()
        weighted_sentances[pred_cat][sent] = logits.max().item()
        
    for ind, weighted_sent in enumerate(weighted_sentances):
        if len(weighted_sent) > 5:
            truncated_sentances = [key for key, val in sorted(weighted_sentances[ind].items(), 
                                                            key=lambda x: x[1], reverse=True)][:5]
            
        else:
            truncated_sentances = [key for key, val in sorted(weighted_sentances[ind].items(), 
                                                            key=lambda x: x[1], reverse=True)]
        
        final_sentances.append(truncated_sentances)
    
    return final_sentances
    
def ext_summarize(text, tokenizer=ext_sum_tokenizer, model=ext_sum_model):
    if not text:
        return

    sentences = [s for s in sent_tokenize(text)]
    article_text = sep_token.join(sentences)

    inputs = tokenizer(
        [article_text],
        max_length=3000,
        padding=True,
        truncation=True,
        return_tensors="pt",
    ).to(device)
    sep_mask = inputs["input_ids"][0] == sep_token_id

    # Fix token_type_ids
    current_token_type_id = 0 
    for pos, input_id in enumerate(inputs["input_ids"][0]):
        inputs["token_type_ids"][0][pos] = current_token_type_id
        if input_id == sep_token_id:
            current_token_type_id = 1 - current_token_type_id

    # Infer model
    with torch.no_grad(): 
        outputs = model(**inputs) 
    logits = outputs.logits[0, :, 1]
    
    logits = logits[sep_mask]
    logits, indices = logits.sort(descending=True)
    logits, indices = logits.cpu().tolist(), indices.cpu().tolist()
    pairs = list(zip(logits, indices))
    pairs = pairs[:3]
    indices = list(sorted([idx for _, idx in pairs]))
    summary = " ".join([sentences[idx] for idx in indices])
    return summary

def generate(tokenizer, model, article_text):
    input_ids = tokenizer(
    [article_text],
    max_length=1200,
    add_special_tokens=True,
    padding=True,
    truncation=True,
    return_tensors="pt"
    )["input_ids"].to(device)

    output_ids = model.generate(
        input_ids=input_ids,
        no_repeat_ngram_size=6,
        
    
    )[0]

    summary = tokenizer.decode(output_ids, skip_special_tokens=True)
    return summary


In [12]:
text_by_cat = [[] for _ in range(6)]

for i in df['full_text']:
    sents_labeled = get_cat(sent_tokenize(i))
    for j in range(6):
        text_by_cat[j].append(ext_summarize(" ".join(sents_labeled[j])))
        

In [13]:
for i in range(6):
    df[f'{i}'] = text_by_cat[i]

In [14]:
df.rename(columns={'0': 'battery' , '1': 'camera', '2': 'screen', '3': 'design', '4': 'hardware', '5': 'sound'}, inplace=True)

In [15]:
df

,file_name,phone_name,blogger,conclusion,full_text,battery,camera,screen,design,hardware,sound
0,asus zenfone 10_text_20230906.txt,asus zenfone 10,4pda,"Как и предшественник, новый ZenFone 10 получи...",Обзор ASUS ZenFone 10: мощный малый глазами за...,"В опциях смартфона есть несколько функций, про...","Во-вторых, адаптивный режим, который автоматич...","Настройки предлагают пять пресетов, регулирующ...",В красном цвете покрытие напоминает наждачную ...,В настройках представлены два варианта интерфе...,И они даже слегка уменьшились по сравнению с п...
1,honor 70_text_20230906.txt,honor 70,4pda,Итоги Honor 70 — интересный смартфон среднего...,Обзор Honor 70: все тренды учтены01.11.2243В р...,Результаты тестирования Стойкий аккумулятор с...,Основной модуль — передовой Sony IMX800 на 54 ...,Традиционный для бренда экран-водопад В Honor ...,В остальном экран только радует. Под экраном р...,На бумаге он может похвастаться новейшей камер...,Звук пишется качественно — в стерео и с грамот...
2,huawei mate 50 pro_text_20230906.txt,huawei mate 50 pro,4pda,Итоги HUAWEI Mate 50 Pro — достойный представ...,Обзор HUAWEI Mate 50 Pro: в погоне за фотоапп...,Есть и беспроводная зарядка Wireless SuperChar...,"А чтобы запечатлеть динамичную сцену днём, выс...",Разве что боковые грани бликуют на солнце — не...,Mate 50 Pro удивляет тыльной крышкой из оранже...,Вся мощь старшего «дракона» Модель работает на...,"Как и положено, аппарат оснастили стереодинами..."
3,huawei mate 50_text_20230906.txt,huawei mate 50,4pda,Итоги HUAWEI Mate 50 — неоднозначный аппарат....,Обзор HUAWEI Mate 50: всё на максималках13.12...,"До 50% гаджет заряжается за 17 минут, а полное...","Хорошая резкость, но не совсем точная цветопер...","У дисплея быстрый отклик, отличные углы обзора...",Насколько удачным получился гаджет? Из коробки...,Mate 50 не стал исключением: он оснащён чипсет...,"Несмотря на это, стереоэффект остаётся ощутимы..."
4,huawei nova 10 pro_text_20230906.txt,huawei nova 10 pro,4pda,"Смартфон радует хорошей производительностью, ...","Обзор HUAWEI nova 10 Pro: снимай красиво, зар...","Не нужны сервисы Google, а скорость подзарядки...",В кадре можно разместить целую компаниюТак выг...,"Набирать текст это не мешает, а вот блики от л...","Это заслуга корпуса толщиной менее 7,9 мм, а т...",Хорош в играх и не только Процессор Qualcomm S...,"Отсутствует и разъём для наушников — впрочем, ..."
...,...,...,...,...,...,...,...,...,...,...,...
9,nothing phone 2_text_20230906.txt,nothing phone 2,wylsacom,"Несмотря на все перечисленные недостатки, Not...",Так как обзор выходит в момент снятия эм...,"Про работу аккумулятора, заявленная ёмкость ко...","Исходя из этого, я рассчитал, что эквивалентно...","Согласно ему, одна или несколько моделей линей...",И если вслед за iPhone 12 многие производители...,Железо Смартфон получил прошлогодний Snapdrago...,"Тут видно, что добавили электронный стаб, а зв..."
10,oneplus 10 pro_text_20230906.txt,oneplus 10 pro,wylsacom,Так стоит ли мечтать о OnePlus 10 Pro или при...,Появление нового флагмана — всегда событие....,Ещё присутствует очень быстрая беспроводная за...,"Хотя можно придраться к тому, что изображение ...","Дисплей с 6,7-дюймовой диагональю, разрешением...","Смартфон слегка покачивается на поверхности, к...",Начинка Для меня это первое устройство с новей...,"Можно выбрать между беззвучным, вибрацией или ..."
11,redmi note 11_text_20230906.txt,redmi note 11,wylsacom,"Пожурить телефон можно за камеру: возможно, к...","Базовый дизайн Мой телефон чёрного цвета,...",Это день активного использования устройства ил...,Разницу в детализации можно увидеть при увелич...,"Сочный экран с 90 Гц У дисплея диагональ 6,43 ...","Я не люблю излишне насыщенные цвета, но в данн...","Но «серые» версии уже продаются, цены начинают...",Во время разговоров можно использовать встроен...
12,tecno pova 5_text_20230906.txt,tecno pova 5,wylsacom,"Когда покупаешь дорогой смартфон, то хочешь э...",Популярная серия смартфонов TECNO POVA пол...,"Если смот

In [20]:
generate(mbart_tokenizer, mbart_model, df[0:5]['conclusion'][4])

c:\Users\al_po\AppData\Local\Programs\Python\Python38\lib\site-packages\transformers\generation\utils.py:1387: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 200 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


'Смартфон HUAWEI nova 10 Pro — это идеальный вариант для тех, кто хочет быстро зарядить батарею и быстро фотографировать.'

In [21]:
ext_summarize(df[0:5]['conclusion'][4])

' Смартфон радует хорошей производительностью, тонким корпусом, качественным экраном, быстрой зарядкой и удобной операционной системой. HUAWEI nova 10 Pro фотографирует лучше многих конкурентов, а прокачанная фронталка станет неоспоримым козырем для любителей автопортретов. Тогда этот аппарат для вас.'

In [22]:
for i in df['conclusion']:
    sents_labeled = get_cat(sent_tokenize(i))
    
    for j in range(6):
        
        text_by_cat[j].append(ext_summarize(" ".join(sents_labeled[j])))

In [23]:
sum_concs = []

for conc in df['conclusion']:
    sum_concs.append(ext_summarize(" ".join(sent_tokenize(conc))))

In [26]:
df['sum_conclusion'] = sum_concs

In [27]:
df

,file_name,phone_name,blogger,conclusion,full_text,battery,camera,screen,design,hardware,sound,sum_conclusion
0,asus zenfone 10_text_20230906.txt,asus zenfone 10,4pda,"Как и предшественник, новый ZenFone 10 получи...",Обзор ASUS ZenFone 10: мощный малый глазами за...,"В опциях смартфона есть несколько функций, про...","Во-вторых, адаптивный режим, который автоматич...","Настройки предлагают пять пресетов, регулирующ...",В красном цвете покрытие напоминает наждачную ...,В настройках представлены два варианта интерфе...,И они даже слегка уменьшились по сравнению с п...,"Как и предшественник, новый ZenFone 10 получи..."
1,honor 70_text_20230906.txt,honor 70,4pda,Итоги Honor 70 — интересный смартфон среднего...,Обзор Honor 70: все тренды учтены01.11.2243В р...,Результаты тестирования Стойкий аккумулятор с...,Основной модуль — передовой Sony IMX800 на 54 ...,Традиционный для бренда экран-водопад В Honor ...,В остальном экран только радует. Под экраном р...,На бумаге он может похвастаться новейшей камер...,Звук пишется качественно — в стерео и с грамот...,Итоги Honor 70 — интересный смартфон среднего...
2,huawei mate 50 pro_text_20230906.txt,huawei mate 50 pro,4pda,Итоги HUAWEI Mate 50 Pro — достойный представ...,Обзор HUAWEI Mate 50 Pro: в погоне за фотоапп...,Есть и беспроводная зарядка Wireless SuperChar...,"А чтобы запечатлеть динамичную сцену днём, выс...",Разве что боковые грани бликуют на солнце — не...,Mate 50 Pro удивляет тыльной крышкой из оранже...,Вся мощь старшего «дракона» Модель работает на...,"Как и положено, аппарат оснастили стереодинами...",Итоги HUAWEI Mate 50 Pro — достойный представ...
3,huawei mate 50_text_20230906.txt,huawei mate 50,4pda,Итоги HUAWEI Mate 50 — неоднозначный аппарат....,Обзор HUAWEI Mate 50: всё на максималках13.12...,"До 50% гаджет заряжается за 17 минут, а полное...","Хорошая резкость, но не совсем точная цветопер...","У дисплея быстрый отклик, отличные углы обзора...",Насколько удачным получился гаджет? Из коробки...,Mate 50 не стал исключением: он оснащён чипсет...,"Несмотря на это, стереоэффект остаётся ощутимы...",Итоги HUAWEI Mate 50 — неоднозначный аппарат....
4,huawei nova 10 pro_text_20230906.txt,huawei nova 10 pro,4pda,"Смартфон радует хорошей производительностью, ...","Обзор HUAWEI nova 10 Pro: снимай красиво, зар...","Не нужны сервисы Google, а скорость подзарядки...",В кадре можно разместить целую компаниюТак выг...,"Набирать текст это не мешает, а вот блики от л...","Это заслуга корпуса толщиной менее 7,9 мм, а т...",Хорош в играх и не только Процессор Qualcomm S...,"Отсутствует и разъём для наушников — впрочем, ...","Смартфон радует хорошей производительностью, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...
9,nothing phone 2_text_20230906.txt,nothing phone 2,wylsacom,"Несмотря на все перечисленные недостатки, Not...",Так как обзор выходит в момент снятия эм...,"Про работу аккумулятора, заявленная ёмкость ко...","Исходя из этого, я рассчитал, что эквивалентно...","Согласно ему, одна или несколько моделей линей...",И если вслед за iPhone 12 многие производители...,Железо Смартфон получил прошлогодний Snapdrago...,"Тут видно, что добавили электронный стаб, а зв...","Несмотря на все перечисленные недостатки, Not..."
10,oneplus 10 pro_text_20230906.txt,oneplus 10 pro,wylsacom,Так стоит ли мечтать о OnePlus 10 Pro или при...,Появление нового флагмана — всегда событие....,Ещё присутствует очень быстрая беспроводная за...,"Хотя можно придраться к тому, что изображение ...","Дисплей с 6,7-дюймовой диагональю, разрешением...","Смартфон слегка покачивается на поверхности, к...",Начинка Для меня это первое устройство с новей...,"Можно выбрать между беззвучным, вибрацией или ...","Судя по OnePlus 10 Pro, идея слияния OPPO и On..."
11,redmi note 11_text_20230906.txt,redmi note 11,wylsacom,"Пожурить телефон можно за камеру: возможно, к...","Базовый дизайн Мой телефон чёрного цвета,...",Это день активного использования устройства ил...,Разницу в детализации можно увидеть при увелич..

In [ ]:
df.fillna("", inplace=True)

In [74]:
df['join_cat'] = df[['camera', 'battery', 'screen', 'design', 'hardware', 'sound']].apply(lambda x: ' '.join(x), axis=1)

In [87]:
df.to_csv('../data/sum_table.csv', encoding='utf-8', index=False)

In [75]:
df['join_cat'].to_csv('../data/categs.csv', encoding='utf-8', index=False)